In [ ]:
import tszip
import sc2ts
import pandas as pd

In [ ]:
ts = tszip.load(snakemake.input[0])
ts

In [ ]:
ti = sc2ts.TreeInfo(ts, show_progress=False)
ti

In [ ]:
ti.plot_mutations_per_node_distribution();

In [ ]:
ti.plot_mutations_per_site_distribution();

In [ ]:
ti.plot_mutations_per_site(annotate_threshold=0.65);

In [ ]:
ti.plot_mutations_per_site(annotate_threshold=0.10, select=ti.mutations_is_reversion);

In [ ]:
ti.plot_deletion_overlaps(annotate_threshold=0.1);

In [ ]:
df = ti.deletions_summary()
df

In [ ]:
df.to_csv(snakemake.output[0]);

In [ ]:
df.length.value_counts()

In [ ]:
dfl = df[df.max_inheritors > 100].sort_values("max_inheritors")
dfl

In [ ]:
dfl.length.value_counts()

In [ ]:
df.sort_values("max_inheritors").tail(20)

# Examine the mutations

In [ ]:
data = []
for site in ts.sites():
    original_mutations = site.metadata["sc2ts"].get("original_mutations", None)
    if original_mutations is not None:
        new_mutations = []
        for mut in site.mutations:
            new_mutations.append(
                {
                    "node": mut.node,
                    "derived_state": mut.derived_state,
                    "metadata": mut.metadata,
                }
            )
        data.append(
            {
                "site": int(site.id),
                "position": int(site.position),
                "original_mutations": original_mutations,
                "new_mutations": new_mutations,
            }
        )

In [ ]:
df_data = []
for row in data:
    A = set((d["node"], d["derived_state"]) for d in row["original_mutations"])
    B = set((d["node"], d["derived_state"]) for d in row["new_mutations"])
    df_data.append({"original_mutations": len(A), "new_mutations": len(B), 
                    "intersection": len(A & B), "position": row["position"]})
    
dfm = pd.DataFrame(df_data)
dfm

In [ ]:
dfm[dfm.original_mutations > 500]

In [ ]:
dfm.original_mutations.hist();

In [ ]:
dfm.new_mutations.hist();

In [ ]:
dfm.original_mutations.describe()

In [ ]:
dfm.new_mutations.describe()

In [ ]:
dfm.intersection.describe()